# LEZIONE 11

**Espressioni regolari**

Finora abbiamo letto file, cercato modelli ed estratto porzioni di righe ritenute interessanti. Abbiamo usato metodi di stringa come split e find, o il frazionamento di elenchi e stringhe per estrarre parti delle righe.

La funzione di ricerca ed estrazione è talmente popolare che è stata sviluppata per Python la libreria espressoni regolari che gestisce con grande eleganza molte di queste attività. La ragione per cui non abbiamo parlato prima delle espressioni regolari è perché, sebbene siano molto potenti, sono un po’ complicate e il padroneggiare la loro sintassi richiede del tempo.

Le espressioni regolari sono quasi un piccolo linguaggio di programmazione dedicato alla ricerca e l’analisi delle stringhe. In effetti sono stati scritti interi libri sul tema delle espressioni regolari. In questo capitolo, parleremo solo delle basi. Per maggiori dettagli sulle espressioni regolari, possiamo consultare:
http://en.wikipedia.org/wiki/Regular_expression
https://docs.python.org/3/library/re.html

La libreria delle espressioni regolari `re` deve essere importata nel nostro programma prima di poterla usare. L’uso più semplice della libreria di espressioni regolari è la funzione search(). Questo programma presenta un uso banale della funzione di ricerca:
```
import re
hand = open('mbox-short.txt')
for line in hand:
    line = line.rstrip()
    if re.search('From:', line):
        print(line)
```

Il file viene aperto e viene letta ogni riga tramite un ciclo in cui viene utilizzata l’*espressione regolare* `search()` per visualizzare solo le righe che contengono la stringa “From:”. Questo programma non sfrutta il vero potere delle espressioni regolari, avremmo potuto usare altrettanto facilmente `line.find()` per ottenere lo stesso risultato.

La potenza delle espressioni regolari viene raggiunta quando utilizziamo i caratteri speciali nella stringa di ricerca che ci consentono di controllare con più precisione quali righe corrispondono alla stringa di ricerca. L’aggiunta di questi caratteri speciali alla nostra espressione regolare ci consente di eseguire abbinamenti ed estrazioni sofisticate tramite la stesura di un codice molto ridotto.

Ad esempio, `^` viene utilizzato nelle espressioni regolari per trovare l’inizio di una riga. Potremmo modificare il nostro script per confrontare solo le righe in cui “From:” si trova all’inizio della riga:

```
import re
hand = open('mbox-short.txt')
for line in hand:
    line = line.rstrip()
    if re.search('^From:', line):
        print(line)
```
Ora lo script lavora solo le righe che iniziano con la stringa “From:”. Questo è un esempio molto elementare che avremmo potuto riprodurre utilizzando il metodo startswith() incluso nella libreria delle stringhe, quindi siamo ancora lontani dallo sfruttare la potenza dell'espressioni regolari, ma serve per introdurre il concetto secondo cui le espressioni regolari possono contenere caratteri speciali che ci danno più controllo su ciò che confronterà l’espressione regolare.




***11.1 Confronto di caratteri nelle espressioni regolari***

Abbiamo a disposizione un certo numero di caratteri speciali che ci permettono di costruire espressioni regolari molto potenti. Il carattere speciale più comunemente usato è il punto (`.`), che corrisponde a un qualsiasi carattere.

Nell’esempio seguente, l’espressione regolare `F..m:` indica una qualsiasi stringa, come ad esempio “From:”, “Fxxm:”, “F12m:” o “F!@m:”, data la presenza di due punti consecutivi (..).

```
# Search for lines that start with 'F', followed by
# 2 characters, followed by 'm:'
import re
hand = open('mbox-short.txt')
for line in hand:
    line = line.rstrip()
    if re.search('^F..m:', line):
        print(line)
```

In [ ]:
import re
print(re.search("^F..m:", "From:aaa@ uct.ac.za"))
print(re.search("^F..m:", "From:aaa@ uct.ac.za").group())

#to avoid DeprecationWarnings
#print(re.search( r"^F..m:", "From:aaa@ uct.ac.za"))
#print(re.search( r"^F..m:", "From:aaa@ uct.ac.za").group())

È una funzione particolarmente potente se combinata con la capacità di indicare che un carattere può essere ripetuto un numero qualsiasi di volte inserendo i caratteri `*` o `+` nell’espressione regolare. Questi caratteri speciali fanno sì che invece di corrispondere a un singolo carattere nella stringa di ricerca, corrispondono a zero o più caratteri (nel caso dell’asterisco) o uno o più caratteri (nel caso del segno
più).

Possiamo ridurre ulteriormente le righe da controllare usando un carattere wild card ripetuto, come nel seguente esempio:

```
# Search for lines that start with From and have an at sign
import re
hand = open('mbox-short.txt')
for line in hand:
    line = line.rstrip()
    if re.search('^From:.+@', line):
        print(line)
```

La stringa di ricerca `^From:.+@` restringe l’analisi alle righe che iniziano con “From:”, seguite da uno o più caratteri (`.+`) e successivamente dal carattere chiocciola, come nell’esempio seguente:

```
From:stephen.marquard@uct.ac.za
```

In [17]:
import re
print(re.search("^From:.+@", "From:a@uct.ac.za"))
print(re.search("^From:.+@", "From:a@uct.ac.za").group())

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


<re.Match object; span=(0, 7), match='From:a@'>
From:a@


Possiamo pensare al carattere wildcard `.+` come all’estensione del controllo di tutti i caratteri inclusi tra i due punti e la chiocciola.

È bene pensare ai caratteri `+` e `*` come "invadenti". Ad esempio, la stringa seguente corrisponderebbe all’ultimo carattere chiocciola mentre `.+` andrebbe oltre, come mostrato di seguito:

In [ ]:
import re
string_to_eval = "From: stephen.marquard@uct.ac.za, csev@umich.edu, e cwen @ iupui.edu"

print(re.search("^From:.+@", string_to_eval).group())

In [ ]:
print(re.search("^From:.+", string_to_eval).group())

***11.2 Estrazione dei dati utilizzando le espressioni
regolari***

Se vogliamo estrarre dei dati da una stringa in Python possiamo usare il metodo `findall()` per selezionare tutte le sottostringhe che corrispondono a un’espressione regolare. Pensiamo, ad esempio, di voler estrarre qualsiasi cosa che assomigli ad un indirizzo email da qualsiasi riga indipendentemente dalla posizione. Ad esempio vogliamo estrarre gli indirizzi email contenuti in ognuna delle seguenti righe:
```
From stephen.marquard@uct.ac.za Sat Jan 5 09:14:16 2008
Return-Path: <postmaster@collab.sakaiproject.org>
for <source@collab.sakaiproject.org>;
Received: (from apache@localhost)
Author: stephen.marquard@uct.ac.za
```
Chiaramente non vogliamo scrivere del codice per ciascun caso suddividendo e segmentando in modo diverso ogni riga, altrimenti che programmatori saremmo? Questo esempio usa `findall()` per trovare le righe contenenti al loro interno indirizzi e-mail e li estraggono.


In [ ]:
import re
s = 'A message from csev@umich.edu to cwen@iupui.edu about meeting @2PM'
lst = re.findall('\S+@\S+', s)
print(lst)

Il metodo `findall()` ricerca la stringa nel secondo argomento e restituisce un elenco di tutte le stringhe che sembrano indirizzi email. Stiamo usando una sequenza di due caratteri che corrisponde a un carattere diverso da spazio (`\S`).

Parafrasando l’espressione regolare, stiamo cercando delle sottostringhe con almeno un carattere diverso da spazio seguito da un carattere chiocciola e da almeno un altro carattere diverso dallo spazio.

L’espressione regolare ha trovato due corrispondenze (`csev@umich.edu` e
`cwen@iupui.edu`), ma ha scartato la stringa `@2PM` perché non sono presenti caratteri diversi dallo spazio prima del carattere chiocciola. Possiamo usare questa espressione regolare in un programma per leggere tutte le righe di un file e visualizzare qualsiasi cosa simile ad un indirizzo email come nell’esempio seguente.
```
# Search for lines that have an at sign between characters
import re
hand = open('mbox-short.txt')
for line in hand:
    line = line.rstrip()
    x = re.findall('\S+@\S+', line)
    if len(x) > 0:
        print(x)
```

Viene letta ogni riga e quindi vengono estratte tutte le sottostringhe che corrispondono alla nostra espressione regolare. Poiché findall() restituisce un elenco, controlliamo se il numero di elementi nel nostro elenco è maggiore di zero per visualizzare solo le righe in cui abbiamo trovato almeno una sottostringa che somigli ad un indirizzo email.



Alcuni degli indirizzi email presentano all’inizio o alla fine caratteri non corretti come `<` o `;`. Ora dobbiamo indicare che siamo interessati solo alle stringhe che iniziano e finiscono con una lettera o un numero.
Per fare ciò sfruttiamo un’altra caratteristica delle espressioni regolari: le parentesi quadre. Queste vengono utilizzate per indicare un insieme di più caratteri accettabili che siamo disposti a considerare corrispondenti. In un certo senso, `'\S'` chiede di confrontare l’insieme di “caratteri diversi dallo spazio”. Ora saremo un po’ più espliciti in termini di caratteri che confronteremo. Ecco la nostra nuova espressione regolare:
```
[a-zA-Z0-9]\S*@\S*[a-zA-Z]
```
Sta diventando tutto più complicato? Ecco perché le espressioni regolari vanno considerate loro stesse un piccolo linguaggio. Traduciamolo come:

cerchiamo sottostringhe che inizino con una singola lettera minuscola o maiuscola o un numero (`[a-zA-Z0-9]`), seguito da zero o più caratteri non vuoti (`\S*`), da una chiocciola, da zero o più caratteri non vuoti (`\S*`), da una lettera maiuscola o minuscola.

Notate che siamo passati da `+` a `*` per indicare zero o più caratteri non vuoti dato che `[a-zA-Z0-9]` è già considerato un carattere non vuoto. Ricordate che `*` o `+` si applicano al singolo carattere immediatamente a sinistra del segno stesso.

Se usiamo questa espressione nel nostro programma, i nostri dati appariranno molto più puliti:
```
# Search for lines that have an at sign between characters
# The characters must be a letter or number
import re
hand = open('mbox-short.txt')
for line in hand:
    line = line.rstrip()
    x = re.findall('[a-zA-Z0-9]\S+@\S+[a-zA-Z]', line)
    if len(x) > 0:
        print(x)
```





Notate che nel caso di “source@collab.sakaiproject.org”, la nostra espressione regolare ha eliminato due caratteri (">;") alla fine delle stringhe. Questo perché quando accodiamo `[a-zA-Z]` alla fine della nostra espressione regolare, vogliamo che qualsiasi stringa trovata debba terminare con una lettera. In altre parole, il carattere ">" viene ignorato perchè il sistema si ferma sull’ultima lettera che rispetti i parametri di ricerca (ovvero "g" è l’ultima corrispondenza valida).
Ricordiamoci inoltre che l’output del programma è un elenco composto da un’unica stringa.

***11.3 Combinare ricerca ed estrazione***

Immagina che si debbano trovare i numeri contenuti nelle righe che iniziano con “X-” come, ad esempio:
```
X-DSPAM-Confidence: 0.8475
X-DSPAM-Probability: 0.0000
```
Insomma, non vogliamo i numeri in virgola mobile provenienti da una qualsiasi riga; vogliamo soltanto i numeri contenuti nelle righe che hanno la sintassi sopra indicata. Possiamo costruire la seguente espressione regolare per selezionare le righe che ci interessano:
`^X-.*: [0-9.]+`

In altre parole, stiamo dicendo alla CPU che stiamo cercando le righe che iniziano con `X-`, seguite da zero o più caratteri (`.*`), da due punti (`:`) e da uno spazio. Dopo lo spazio, cerchiamo uno o più caratteri numerici (0-9) o un punto `[0-9.]+`. Notate che all’interno delle parentesi quadre il punto corrisponde esattamente a un punto (cioè non è un carattere wildcard).

Questa è un’espressione molto stringente che corrisponderà quasi solo alle righe che ci interessano:
```
# Search for lines that start with 'X' followed by any non
# whitespace characters and ':'
# followed by a space and any number.
# The number can include a decimal.
import re
hand = open('mbox-short.txt')
for line in hand:
    line = line.rstrip()
    if re.search(r'^X\S*: [0-9.]+', line):
        print(line)
```





Eseguendo il programma, vedremo che verranno visualizzate solo le righe che stiamo cercando.

Ora dobbiamo risolvere il problema dell’estrazione dei numeri. Anche se sarebbe abbastanza semplice usare split, possiamo sfruttare un’altra caratteristica delle espressioni regolari per cercare e analizzare allo stesso tempo le righe.

Le parentesi sono considerate un altro carattere speciale nelle espressioni regolari. Se aggiungiamo le parentesi a un’espressione regolare, queste vengono ignorate durante le operazioni di ricerca delle stringhe. Se però utilizziamo `findall()`, le parentesi indicano che non vogliamo solo effettuare una ricerca, ma vogliamo estrarre la porzione di
riga che corrisponde all’espressione regolare.

```
# Search for lines that start with 'X' followed by any
# non whitespace characters and ':' followed by a space
# and any number. The number can include a decimal.
# Then print the number if it is greater than zero.
import re
hand = open('mbox-short.txt')
for line in hand:
    line = line.rstrip()
    x = re.findall(r'^X\S*: ([0-9.]+)', line)
    if len(x) > 0:
        print(x)
```
Invece di utilizzare `search()`, aggiungendo le parentesi attorno alla parte dell’espressione regolare che rappresenta il numero in virgola mobile, vogliamo che `findall()` ci restituisca solo la parte in virgola mobile della stringa confrontata.

Anche se i numeri sono ancora parte di un elenco e devono essere convertiti da stringhe a numeri in virgola mobile, grazie al potere delle espressioni regolari abbiamo potuto cercare ed estrarre le informazioni che ci interessano.
Facciamo un altro esempio di questa tecnica: se diamo un’occhiata al file, vedremo un certo numero di righe simili alla seguente:

`Details: http://source.sakaiproject.org/viewsvn/?view=rev&rev=39772`

Se volessimo estrarre tutti i numeri delle versioni (i numeri interi alla fine della riga) utilizzando la tecnica precedente, potremmo scrivere lo script seguente:


```
# Search for lines that start with 'Details: rev='
# followed by numbers
# Then print the number if one is found
import re
hand = open('mbox-short.txt')
for line in hand:
    line = line.rstrip()
    x = re.findall('^Details:.*rev=([0-9]+)', line)
    if len(x) > 0:
        print(x)
```
Traduciamo la nostra espressione regolare: stiamo cercando le righe che iniziano con "Details:", seguito da un numero qualsiasi di caratteri (`.*`), seguito da "rev=", e quindi da una o più cifre. Vogliamo trovare le righe che combacino con l’intera espressione ma deve essere estratto il numero intero presente alla fine della riga, come indicato da `[0-9] +` racchiuso tra parentesi.

Ricordiamo che `[0-9]+` è “avido” e tenta di includere quante più cifre nella stringa prima di estrarla. Questo comportamento è il motivo per cui otteniamo tutte e cinque le cifre per ogni numero. La libreria di espressioni regolari si espande in entrambe le direzioni fino a quando non incontra un carattere non numerico o l’inizio o la fine di una riga.
Ora possiamo utilizzare le espressioni regolari per rifare un esercizio visto in precedenza in cui eravamo interessati all’ora del giorno di ciascuna mail. Abbiamo cercato questo tipo di righe:

`From stephen.marquard@uct.ac.za Sat Jan 5 09:14:16 2008`

ed in ognuna abbiamo estratto l’ora del giorno. In precedenza abbiamo utilizzato due chiamate a `split`: per prima cosa la riga è stata divisa in parole e poi abbiamo estratto la quinta parola e diviso nuovamente la riga al carattere due punti per tirare fuori i due caratteri che ci interessavano. Nonostante abbia funzionato, in realtà il nostro codice è piuttosto fragile: dato che presuppone che le righe siano ben formattate. Se dovessimo aggiungere un numero congruo di controlli degli errori (o un grande blocco try/except) per assicurarci che il programma non si
blocchi quando incontra righe formattate in modo errato, il codice si espanderebbe di almeno 10-15 righe diventando piuttosto difficile da leggere.


Possiamo farlo in un modo molto più semplice tramite la seguente espressione regolare:

`^From .* [0-9][0-9]:`

Con questa espressione regolare stiamo cercando le righe che iniziano con “From” (notate lo spazio), seguito da un numero qualsiasi di caratteri (`.*`), poi da uno spazio, da due cifre `[0-9][0-9]`, ed infine un carattere di due punti. Questa però è solo la definizione del tipo di righe che stiamo cercando. Per estrarre l’ora abbiamo bisogno di utilizzare `findall()`, e di aggiungere le parentesi attorno alle due cifre come indicato qui sotto:

`^From .* ([0-9][0-9]):`

Ora possiamo scrivere il nostro script:
```
# Search for lines that start with From and a character
# followed by a two digit number between 00 and 99 followed by ':'
# Then print the number if one is found
import re
hand = open('mbox-short.txt')
for line in hand:
    line = line.rstrip()
    x = re.findall('^From .* ([0-9][0-9]):', line)
    if len(x) > 0: print(x)
```





***11.4 Carattere Escape***

Dato che utilizziamo caratteri speciali nelle espressioni regolari per indicare l’inizio o la fine di una riga o specificare i caratteri wild card, abbiamo bisogno di un modo per indicare che questi sono caratteri “normali” e vogliamo confrontarli con caratteri reali come il segno di dollaro o un accento circonflesso. Possiamo indicare questa intenzione anteponendo backslash al carattere che ci interessa. Ad esempio, possiamo individuare gli importi in denaro tramite la seguente espressione regolare.

In [ ]:
import re
x = 'We just received $10.00 for cookies.'
y = re.findall('\$[0-9.]+',x)

print(y)

Dal momento che facciamo precedere backslash al simbolo del dollaro, questo
verrà interpretato come simbolo del dollaro anziché “fine della riga”. Il resto
dell’espressione regolare corrisponderà a una o più cifre o al carattere del punto.

**N.B.**: All’interno di parentesi quadre, i caratteri non sono considerati “speciali”.

Quindi quando diciamo `[0-9.]`, indichiamo effettivamente numeri o il punto.

- **Al di fuori delle parentesi quadre, il punto è visto come un carattere "wild card" che corrisponde a qualsiasi carattere**
- **All’interno delle parentesi quadre il punto è solo un punto.**

***11.5 Sommario***

Anche se abbiamo solo scalfito la superficie del mondo delle espressioni regolari, ora abbiamo un po’ imparato la struttura del loro linguaggio. Sono *stringhe di ricerca contenenti caratteri speciali che comunicano i nostri desideri al sistema delle espressioni regolari definendo cosa è "coincidente" e cosa va estratto dalle stringhe corrispondenti*.

Ecco alcuni di quei caratteri speciali e sequenze di caratteri:
- ^ Corrisponde all’inizio riga.
- $ Corrisponde alla fine riga.
- . Corrisponde a qualsiasi carattere (un carattere wild-card).
- \s corrisponde a uno spazio.
- \S corrisponde a un carattere diverso dallo spazio (opposto a \s).
- "*" Si applica al carattere immediatamente precedente e indica zero o più dei ripetizioni dello stesso.
- *? Si applica al carattere immediatamente precedente e indica zero o più dei
ripetizioni dello stesso in "modalità non avida".
- "+" Si applica al carattere immediatamente precedente e indica uno o più dei caratteri precedenti.
- +? Si applica al carattere immediatamente precedente e indica uno o più dei
caratteri precedenti in “modalità non avida”.
- [aeiou] Corrisponde a un singolo carattere fintanto che quel carattere si trova nell’insieme specificato. In questo esempio, ricercherebbe “a”, “e”, “i”, “o” oppure “u” e nessun altro carattere.
- [a-z0-9] È possibile specificare intervalli di caratteri utilizzando il segno meno (-). In questo esempio si tratta di un singolo carattere che deve essere una lettera minuscola o una cifra.
- [ˆA-Za-z] Quando il primo carattere nella notazione è un accento circonflesso, si inverte la logica. In questo esempio viene ricercato un qualsiasi singolo carattere diverso da una lettera maiuscola o minuscola.
- () Quando le parentesi vengono aggiunte a un’espressione regolare vengono ignorate ai fini della ricerca, ma consentono di estrarre un particolare sottoinsieme della stringa desiderata, diversamente da quanto accade con findall() dove viene considerata l’intera riga.
- \b Corrisponde alla stringa vuota ma solo all’inizio o alla fine di una parola.
- \B Corrisponde alla stringa vuota ma non all’inizio o alla fine di una parola.
- \d Corrisponde a qualsiasi cifra decimale compresa nell’insieme [0-9].
- \D Corrisponde a qualsiasi carattere non numerico ed è equivalente al set [ˆ0-9].

***11.6 Sezione bonus per utenti Unix/Linux***

Il supporto per la ricerca di file tramite espressioni regolari è stato incorporato nel sistema operativo Unix sin dagli anni ’60 ed è disponibile, in una forma o in un’altra, in quasi tutti i linguaggi di programmazione.

Esiste infatti un programma a riga di comando incorporato in Unix chiamato grep
(Generalized Regular Expression Parser) che funziona più o meno come mostrato
negli esempi di search() di questo capitolo. Quindi se usiamo un sistema Macintosh o Linux, possaimo provare questo comando nella finestra della riga di comando:

```
$ grep '^From:' mbox-short.txt
From: stephen.marquard@uct.ac.za
From: louis@media.berkeley.edu
From: zqian@umich.edu
From: rjlowe@iupui.edu
```
Grep mostra le righe che iniziano con la stringa “From:” presenti nel file
mbox-short.txt. Quando faremo un po’ di pratica con il comando grep e studieremo
la sua documentazione, noteremo alcune sottili differenze tra il funzionamento
delle espressioni regolari in Python e in grep. Ad esempio, grep non supporta
il carattere non vuoto “§”, quindi per ottenere lo stesso risultato dovremo usare una notazione per gli insiemi leggermente più complessa: “[‘ˆ’]” che indica di individuare un carattere diverso da uno spazio.


***11.7 Debug***

Python è dotato di una documentazione integrata semplice e rudimentale che può
essere molto utile se abbiamo bisogno di un rapido aggiornamento per rinfrescare la memoria sul nome di un particolare metodo. Questa documentazione può essere visualizzata in modalità interattiva nell’interprete Python. Tramite il comando help().

In [ ]:
#insert "modules" in the interactive box
help()

Se sappiamo già quale modulo utilizzare, possiamo usare il comando dir() per ottenere un elenco dei metodi disponibili nel modulo:

In [ ]:
import re
dir(re)

Utilizzando il comando dir è inoltre possibile ottenere una documentazione ridotta su uno specifico metodo.

In [ ]:
help (re.search)

La documentazione integrata non è molto ampia ma può essere comunque utile
quando abbiamo fretta o non abbiamo accesso a un browser Web o ad un motore di ricerca.

***11.8 Glossario***

- **Codice fragile** Codice che funziona quando i dati di input sono in un formato particolare ma è soggetto a malfunzionamenti se c’è qualche variazione rispetto al formato corretto. Lo chiamiamo “codice fragile” perché si “rompe”
facilmente.
- **Corrispondenza avida** La nozione per indicare che in un’espressione regolare i caratteri “+” e “*” si espandono verso l’esterno per corrispondere alla stringa più grande possibile.
- **Grep** Comando disponibile nella maggior parte dei sistemi Unix che permette la ricerca nei file di testo di righe che soddisfino le espressioni regolari impostate dall’utente. Il nome del comando è l’acronimo di “Generalized Regular Expression Parser”.
- **Espressione regolare** Linguaggio per impostare ricerche complesse di stringhe. Un’espressione regolare può contenere caratteri speciali che indicano parametri come il focalizzare una ricerca solo all’inizio o alla fine di una riga e/o molte altre funzionalità simili.
- **Wild-card** Un carattere speciale che indica qualsiasi carattere. Nelle espressioni regolari il carattere wild-card è il punto (.).